In [ ]:
from fastai.metrics import accuracy
from fastai.optimizer import OptimWrapper
from fastai.vision.all import *
from PIL import Image

from torch import optim
import torch.nn as nn
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import AdamW
import cv2

import gc
import glob
import inspect
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from functools import partial

from skimage.metrics import structural_similarity as ssim
from skimage import color, transform
from sklearn.model_selection import StratifiedKFold

import warnings 
warnings.filterwarnings('ignore')
set_seed(3, reproducible=True)

In [ ]:
trn_path = '/kaggle/input/zindidata/images/images/'
files    = get_image_files(trn_path) 

In [ ]:
train            = pd.read_csv('/kaggle/input/zindidata/Train.csv')
test             = pd.read_csv('/kaggle/input/zindidata/Test.csv')
SampleSubmission = pd.read_csv('/kaggle/input/zindidata/SampleSubmission.csv')

In [ ]:
Train = train.copy()
Test = test.copy()

skf = StratifiedKFold(5, shuffle=True, random_state=3)
X = Train.drop(columns='damage')
y = Train.damage

for fold, (_, valid_index) in enumerate(skf.split(X, y)):
  Train.loc[valid_index, "fold"] = fold

Train.fold = Train.fold.astype(int)

In [ ]:
damage = Train.damage.unique()

In [ ]:
class_weights = torch.FloatTensor([1.3, 1.2, 1.4, 1.0, 2.8]).cuda()

In [ ]:
IMG_HEIGHT = IMG_WIDTH = 224
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
INCEPTION_MEAN = INCEPTION_STD = (0.5, 0.5, 0.5)


class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        
        return aug_img


def get_train_transforms(mean_std):
    augmentations = [
        A.VerticalFlip(p=.4),
        A.HorizontalFlip(p=.5),
        A.RandomRotate90(p=.2),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.3, 
            rotate_limit=45, border_mode=0, p=.4
        ),
        A.RandomBrightnessContrast(brightness_limit=(-0.15,0.2), contrast_limit=(-0.1, 0.1), p=0.5),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT*0.2),
            max_w_size=int(IMG_WIDTH*0.2),
            num_holes=4,
            p=.7,
        ),
    ]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(mean_std):
    augmentations = []
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)

def get_item_tfms(mean_std='imagenet'):
    return [Resize(224, method='squish'), AlbumentationsTransform(get_train_transforms(mean_std), get_valid_transforms(mean_std))]

In [ ]:
def train(arch, item, batch, epoch=7):
    m_name = 'V'
    dblock = DataBlock(blocks = (ImageBlock, CategoryBlock(vocab=damage)),
                       get_x = ColReader('filename', pref = trn_path),
                       get_y = ColReader('damage'),
                       splitter = RandomSplitter(valid_pct = 0.2, seed = 3),
                       item_tfms = item, #Resize(320, method = 'squish'),
                       batch_tfms = batch, #aug_transforms(size = 128, min_scale = 0.75)
                      )

    dls = dblock.dataloaders(Train, bs=98)
    learn = vision_learner(dls, arch, loss_func = LabelSmoothingCrossEntropyFlat(weight=class_weights),
                           metrics=[accuracy, error_rate, Precision(average='macro'), Recall(average='macro'), RocAuc(average='macro')]).to_fp16()
    learn.fine_tune(epoch, 0.018)
    dls.rng.seed(3)
    interp = ClassificationInterpretation.from_learner(learn)
    losses,idxs = interp.top_losses()
    len(dls.valid_ds)==len(losses)==len(idxs)
    interp.plot_confusion_matrix(figsize=(7,7))

    return learn

In [ ]:
def get_dls(fold=0, mean_std='imagenet'):
    clas_block = DataBlock(blocks = (ImageBlock, CategoryBlock(vocab=damage)),
                       get_x = ColReader('filename', pref = trn_path),
                       get_y = ColReader('damage'),
                       splitter = splitter=MaskSplitter(Train.fold == fold),
                       item_tfms = get_item_tfms(mean_std), #Resize(320, method = 'squish'),
                       batch_tfms = aug_transforms(min_scale= 0.75), #aug_transforms(size = 128, min_scale = 0.75)
                      )
    dls = dblock.dataloaders(Train, bs=32)
    dls.rng.seed(21)

    return dls

In [ ]:
def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
predictions = []
for fold in range(5):
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'fold-{fold}'
    dls = get_dls(fold, 'imagenet')
    learn = vision_learner(dls, 'convit_small.fb_in1k', 
                           loss_func = CrossEntropyLossFlat(weight=class_weights),
                           metrics=[accuracy, error_rate, Precision(average='macro'), Recall(average='macro'), RocAuc(average='macro')])
    
    learn.fine_tune(5, 0.075)
    
    interp = ClassificationInterpretation.from_learner(learn)
    losses,idxs = interp.top_losses()
    len(dls.valid_ds)==len(losses)==len(idxs)
    interp.plot_confusion_matrix(figsize=(7,7))
    
    valid = learn.dls.valid
    preds1 , targets = learn.tta(dl=valid, n=4)
    print(accuracy(preds1 , targets))
    
    test_dl = learn.dls.test_dl('/kaggle/input/zindidata/images/images/' + Test['filename'])
    preds1 , _ = learn.tta(dl=test_dl)
    preds1 = F.softmax(preds1, dim=1)
    predictions.append(preds1)

    free_memory([model, learn, preds1, dls])

In [ ]:
for i in range(5):
    pred1 += predictions[i]
pred1 /= 5

In [ ]:
submission_sf = pd.DataFrame({
    
    'ID': Test['ID'],
    'DR': preds1[:, 0].squeeze().numpy(),
    'G' : preds1[:, 1].squeeze().numpy(),
    'ND': preds1[:, 2].squeeze().numpy(),
    'WD': preds1[:, 3].squeeze().numpy(),
    'other': preds1[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('convitfold.csv', index=False)

In [ ]:
submission_sf